In [1]:
import pandas as pd
import numpy as np

In [2]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\SREEKALA\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
import re


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SREEKALA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SREEKALA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SREEKALA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Loading the new smishing data set

In [5]:
filepath = 'analysisdataset.csv'
smishdata = pd.read_csv(filepath, encoding = 'ISO-8859-1')

In [81]:
print(smishdata.head())

   messageid                                           Fulltext  \
0          3  Text Message\nThu, Jul 29, 19:10\nCostco: Dani...   
1          5  <\n+1 (872) 279-0672 >\nText Message\nWed, Feb...   
2          6  <\n+1 (806) 224-7886 >\nText Message\nThu, Sep...   
3          7  Text Message\nToday 2:30 PM\nwho played golf\n...   
4          8  (8\n+1 (775) 537-4497\ncanador to them\nTato m...   

              Sender    SenderType          timeReceived  \
0              42003    Short Code  03/31/2022, 21:58:50   
1  +1 (872) 279-0672  Phone Number  04/02/2022, 02:59:56   
2  +1 (806) 224-7886  Phone Number  04/02/2022, 03:03:00   
3                NaN           NaN  04/02/2022, 23:42:38   
4  +1 (775) 537-4497  Phone Number  04/02/2022, 23:46:34   

                                            MainText                    Url  \
0  Costco: Daniel, the code 42003 printed on your...  f2gpy.info/RzNKEwsZve   
1  Hi, you still owe UPS $4.10 USD in customs fee...                    NaN   

# EDA

In [6]:
# Columns to be dropped
columns_to_drop = [
    'URL Subcategory',
    'FullyQualifiedDomain',
    'Domain Registrar',
    'Domain Creation Date',
    'Domain Last Update',
    'timeReceived',
    'Subdomain',
    'RedirectedURL',
    'Url',
    'Fulltext','messageid','Sender','SenderType','Domain','TLD','Brand','Message Categories'
]

# Drop the specified columns
smishdata.drop(columns_to_drop, axis=1, inplace=True)

In [7]:
# Remove rows with null values
smishdata.dropna(inplace=True)


In [8]:
# Display the first few rows to verify the columns are removed
print(smishdata.head())

                                             MainText  Detected  Malicious  \
0   Costco: Daniel, the code 42003 printed on your...       0.0        0.0   
2   wel01.us/r/rest05 WELLS FARGO(CS):Profile lock...       0.0        0.0   
6   Dear. You are invited to join the (Bitcoin) in...       0.0        0.0   
7   CITI: Unauthorized activity was detected, we h...       0.0        0.0   
10  BofA:Your account has been restricted, visit s...       0.0        0.0   

    Phishing  Suspicious  Malware  
0        0.0         0.0      0.0  
2        0.0         0.0      0.0  
6        0.0         0.0      0.0  
7        0.0         0.0      0.0  
10       0.0         0.0      0.0  


In [9]:
#Figuring out use for phishing, suspicious and malware columns

columns_of_interest = ['Phishing', 'Suspicious', 'Malware']

# Checking for unique values and their counts in the columns of interest
unique_values_counts = {col: smishdata[col].value_counts() for col in columns_of_interest}
unique_values_counts

{'Phishing': Phishing
 0.0     559
 1.0     122
 2.0      42
 3.0      30
 14.0     29
 13.0     21
 4.0      19
 5.0      18
 10.0     16
 6.0      13
 15.0     13
 9.0      12
 7.0      12
 17.0      8
 8.0       8
 11.0      4
 12.0      2
 16.0      1
 18.0      1
 Name: count, dtype: int64,
 'Suspicious': Suspicious
 0.0    850
 1.0     78
 2.0      2
 Name: count, dtype: int64,
 'Malware': Malware
 0.0    805
 1.0     67
 3.0     27
 2.0     24
 5.0      5
 6.0      1
 4.0      1
 Name: count, dtype: int64}

In [10]:
# Scam messages as those that have a value of 1 or more in any of the Phishing, Suspicious, or Malware columns
smishdata['IsScam'] = (smishdata['Phishing'] > 0) | (smishdata['Suspicious'] > 0) | (smishdata['Malware'] > 0)



# Calculate the count of scam and non-scam messages
scam_counts = smishdata['IsScam'].value_counts()

scam_counts

IsScam
False    511
True     419
Name: count, dtype: int64

NON SCAM MESSAGES - 511

SCAM MESSAGES - 419

In [11]:
columns_to_drop2 =['Phishing','Suspicious','Malware','Detected','Malicious']
smishdata.drop(columns_to_drop2, axis=1, inplace=True)


# Preprocessing

In [12]:
# Initialize the lemmatizer and stopwords list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [13]:
def clean_lemmatize_remove_stopwords(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove special characters and punctuation using regex
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize the cleaned text
    word_tokens = word_tokenize(cleaned_text)
    
    # Filter out the stopwords and lemmatize the remaining words
    filtered_lemmatized_text = [lemmatizer.lemmatize(word) for word in word_tokens if word not in stop_words]
    
    # Rejoin words into a string and return
    return ' '.join(filtered_lemmatized_text)

In [14]:
smishdata['MainText'] = smishdata['MainText'].apply(clean_lemmatize_remove_stopwords)


In [59]:
smishdata.head()

,MainText,IsScam
0,costco daniel code 42003 printed receipt 10 ca...,False
2,wel01usrrest05 well fargocsprofile locked unus...,False
6,dear invited join bitcoin internal discussion ...,False
7,citi unauthorized activity detected moved fowa...,False
10,bofayour account restricted visit secbofapagel...,False


### Let's now load the UCI data set

In [15]:
sms_text = pd.read_csv("spam.csv", encoding='latin-1')

In [16]:
sms_text.dropna(how="any", inplace=True, axis=1)
sms_text.columns = ['label', 'message']

In [95]:
sms_text.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
sms_text['cleaned_message'] = sms_text['message'].apply(clean_lemmatize_remove_stopwords)

In [18]:
sms_text['cleaned_message'].iloc[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

## Label Encoding

In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
sms_text['label_encoded'] = le.fit_transform(sms_text['label'])
smishdata['IsScam_encoded'] = le.fit_transform(smishdata['IsScam'])

In [99]:
sms_text.head()

,label,message,cleaned_message,label_encoded
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,0
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,1
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf life around though,0


In [20]:

smishdata['IsScam_encoded'].value_counts()

IsScam_encoded
0    511
1    419
Name: count, dtype: int64

In [21]:
smishdata.drop('IsScam', axis = 1, inplace=True)
sms_text.drop(['message','label'],axis = 1, inplace=True)

Renaming the column names before concatenation

In [22]:
smishdata.rename(columns={'MainText': 'sms', 'IsScam_encoded': 'label'}, inplace=True)
sms_text.rename(columns={'cleaned_message': 'sms', 'label_encoded': 'label'}, inplace=True)

In [23]:
sms_df =  pd.concat([smishdata, sms_text], ignore_index=True)

In [24]:
sms_df['label'].value_counts()

label
0    5336
1    1166
Name: count, dtype: int64

In [25]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA Available:", cuda_available)




CUDA Available: True


### BERT Tokenizer

In [27]:
%pip install transformers

     ---------------------------------------- 9.0/9.0 MB 5.2 MB/s eta 0:00:00
     -------------------------------------- 152.8/152.8 KB 8.9 MB/s eta 0:00:00
     -------------------------------------- 388.9/388.9 KB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 4.9 MB/s eta 0:00:00
     -------------------------------------- 287.9/287.9 KB 5.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\SREEKALA\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [28]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel


c:\Users\SREEKALA\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

c:\Users\SREEKALA\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SREEKALA\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


### Define Dataset

In [30]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


### Define RNN Classifier

In [31]:
class RNNClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super(RNNClassifier, self).__init__()
        self.bert = bert_model
        embedding_dim = bert_model.config.to_dict()['hidden_size']
        self.rnn = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            bidirectional=bidirectional,
            dropout=dropout,
            batch_first=True
        )
        self.output = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            embedded = self.bert(input_ids, attention_mask=attention_mask)[0]
        _, (hidden, _) = self.rnn(embedded)
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        output = self.output(hidden)
        return output


In [37]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available, using CPU.")

Using CUDA: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [38]:
model = RNNClassifier(bert, hidden_dim=256, output_dim=2, n_layers=2, bidirectional=True, dropout=0.1)
model = model.to(device)
#criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


### Train - Test Split

In [40]:
from sklearn.model_selection import train_test_split
# Extract features and target
texts = sms_df['sms'].tolist()
labels = sms_df['label'].tolist()

# Split data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.30, random_state=42
)


### Create DataLoaders

In [41]:
# Create Dataset objects
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)

# Create Data Loaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [42]:
def train(model, criterion, optimizer, dataloader, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, criterion, dataloader, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            total_correct += (preds == labels).sum().item()
    return total_loss / len(dataloader), total_correct / len(dataloader.dataset)


In [43]:
# Parameters
epochs = 4
criterion = nn.CrossEntropyLoss().to(device)

for epoch in range(epochs):
    train_loss = train(model, criterion, optimizer, train_loader, device)
    test_loss, test_accuracy = evaluate(model, criterion, test_loader, device)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}')


Epoch 1, Train Loss: 0.1876, Test Loss: 0.1749, Test Accuracy: 0.93
Epoch 2, Train Loss: 0.1517, Test Loss: 0.1530, Test Accuracy: 0.93
Epoch 3, Train Loss: 0.1252, Test Loss: 0.1755, Test Accuracy: 0.93
Epoch 4, Train Loss: 0.1084, Test Loss: 0.1397, Test Accuracy: 0.93


In [44]:
torch.save(model.state_dict(), 'model_state_dict.pth')
